## Лабораторная работа № 2 
## Машинное обучение на больших данных с использованием фреймворка Apache Spark и библиотеки SparkML

### Часть 2

В данной части работы рассмотрены:
* подготовка признаков для рeшения задачи **RandomForestClassifier**;
* создание и обучение модели;
* оценка качества модели.

#### Запуск `Spark`-сессии

Подключаем необходимые библиотеки.

In [47]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.ml.feature import VectorAssembler, StringIndexer, Binarizer, Bucketizer
from pyspark.ml.functions import vector_to_array
from pyspark.ml.classification import GBTClassifier, GBTClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql import Window


from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Сформируем объект конфигурации для `Apache Spark`, указав необходимые параметры.

In [48]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 2 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [49]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`. В процессе создания сессии происходит подключение к кластеру `Apache Hadoop`, что может занять некоторое время.

In [50]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

24/11/17 20:17:47 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/11/17 20:18:07 WARN Client: Same path resource file:///home/user0/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.5_2.12-1.6.0.jar added multiple times to distributed cache.


#### Загрузка датасета

Укажем базу данных, которая была создана в первой лабораторной работе.

In [51]:
database_name = "gysynin_dmitry_database"

Установим созданную базу данных как текущую.

In [52]:
spark.catalog.setCurrentDatabase(database_name)

Прочитаем таблицу с **предобработанным датасетом** и загрузим её в `Spark Dataframe`.

In [53]:
df = spark.table("sobd_lab1_processed_table")

Выведем прочитанную таблицу на экран.

In [31]:
df.show()

24/11/17 20:10:40 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/11/17 20:10:55 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+--------+---------------+-------------+----------+------------------+------------+-----------+-----+----------+------------+------------+
|VendorID|passenger_count|trip_distance|RateCodeID|store_and_fwd_flag|payment_type|fare_amount|extra|tip_amount|tolls_amount|total_amount|
+--------+---------------+-------------+----------+------------------+------------+-----------+-----+----------+------------+------------+
|       2|              1|         2.72|         1|             false|           1|       12.5|  0.5|      3.25|         0.0|       17.05|
|       2|              1|         1.59|         1|             false|           1|        7.5|  0.5|       1.6|         0.0|        10.4|
|       2|              1|          0.9|         1|             false|           1|        6.0|  0.5|       1.0|         0.0|         8.3|
|       2|              1|         1.96|         1|             false|           1|        9.5|  0.5|       2.0|         0.0|        12.8|
|       2|              3| 

Вспомним описание столбцов и параметры датасета, проанализированные в первой лабораторной работе.

| Название столбца  | Расшифровка |
| ------------- | ------------- |
| VendorID | Код, указывающий на поставщика услуг TPEP, который предоставил запись. 1 - Creative Mobile Technologies 2- VeriFone Inc. |
| passenger_count | Количество пассажиров |
| trip_distance | Расстояние поездки |
| RateCodeID | Окончательный код тарифа, действующий в конце поездки. 1- Стандартный тариф 2- JFK 3- Ньюарк 4- Нассау или Вестчестер 5- Тариф по договоренности 6- Групповая поездка |
| store_and_fwd_flag | Этот флаг указывает, хранилась ли запись о поездке в памяти автомобиля перед отправкой поставщику, так называемый «store and forward», поскольку автомобиль не имел соединения с сервером. Y = поездка с сохранением и пересылкой N= не поездка с сохранением и пересылкой |
| payment_type | Цифровой код, обозначающий, каким образом пассажир оплатил поездку. 1- Кредитная карта 2- Наличные 3- Без оплаты 4-Спор 5- Неизвестно 6-Анулированная поездка |
| fare_amount | Тариф за время и расстояние, рассчитанный счетчиком. |
| extra | Различные доплаты и надбавки. В настоящее время сюда входят только сборы в размере 0,50 и 1 доллара в час пик и ночные сборы. |
| tip_amount | Сумма чаевых - Это поле автоматически заполняется для чаевых по кредитной карте. Чаевые наличными не учитываются. |
| tolls_amount | Общая сумма всех дорожных сборов, оплаченных в поездке. |
| total_amount | Общая сумма, взимаемая с пассажиров. Не включает денежные чаевые. |

Вспомним схему данных.

In [32]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



Вычислим количество строк в датафрейме.

In [33]:
df.count()

6036111

#### Постановка задачи

Для датасета, заданного представленными колонками, требуется построить модель **Random Forest Classifier** для оценки факта того, **много или мало будет пассажиров в данной поездке**, по всем остальным признакам.

Для этого бинаризируем столбец passenger_count по границе 5. 0-4 пассажиров будет 0 (мало), а 5-9 пассажиров будет 1 (много)

Для оценки качества обучения следует использовать метрики `Precision` и `Recall`. Оценить максимально возможное значение **точности** при полноте не менее 60%.

#### Подготовка признаков

In [54]:
df = df.withColumn("trip_distance", F.col("trip_distance").cast(DoubleType()))
df = df.withColumn("fare_amount", F.col("fare_amount").cast(DoubleType()))
df = df.withColumn("total_amount", F.col("total_amount").cast(DoubleType()))
df = df.withColumn("tip_amount", F.col("tip_amount").cast(DoubleType()))

In [55]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: float (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: double (nullable = true)



In [56]:
from pyspark.ml.feature import Bucketizer

In [57]:
df = df.withColumn("passenger_count", F.col("passenger_count").cast(DoubleType()))
splits = [-float("inf"), 5.0, float("inf")]

In [ ]:
# df = df.drop('passenger_count_binary')

In [58]:
bucketizer = Bucketizer(splits=splits, inputCol="passenger_count", outputCol="passenger_count_binary")
df = bucketizer.transform(df)
df.show()

24/11/17 20:18:55 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


+--------+---------------+------------------+----------+------------------+------------+-----------+-----+------------------+------------+------------------+----------------------+
|VendorID|passenger_count|     trip_distance|RateCodeID|store_and_fwd_flag|payment_type|fare_amount|extra|        tip_amount|tolls_amount|      total_amount|passenger_count_binary|
+--------+---------------+------------------+----------+------------------+------------+-----------+-----+------------------+------------+------------------+----------------------+
|       2|            1.0|2.7200000286102295|         1|             false|           1|       12.5|  0.5|              3.25|         0.0|17.049999237060547|                   0.0|
|       2|            1.0| 1.590000033378601|         1|             false|           1|        7.5|  0.5| 1.600000023841858|         0.0|10.399999618530273|                   0.0|
|       2|            1.0|0.8999999761581421|         1|             false|           1|       

In [59]:
df = df.withColumn("passenger_count_binary", F.col("passenger_count_binary").cast(IntegerType()))

In [60]:
df.show()

+--------+---------------+------------------+----------+------------------+------------+-----------+-----+------------------+------------+------------------+----------------------+
|VendorID|passenger_count|     trip_distance|RateCodeID|store_and_fwd_flag|payment_type|fare_amount|extra|        tip_amount|tolls_amount|      total_amount|passenger_count_binary|
+--------+---------------+------------------+----------+------------------+------------+-----------+-----+------------------+------------+------------------+----------------------+
|       2|            1.0|2.7200000286102295|         1|             false|           1|       12.5|  0.5|              3.25|         0.0|17.049999237060547|                     0|
|       2|            1.0| 1.590000033378601|         1|             false|           1|        7.5|  0.5| 1.600000023841858|         0.0|10.399999618530273|                     0|
|       2|            1.0|0.8999999761581421|         1|             false|           1|       

In [61]:
# Выберем нужные столбцы
feature_columns = ["VendorID", "RateCodeID", "payment_type", "store_and_fwd_flag", "fare_amount", 
                   "extra", "tip_amount", "tolls_amount", "total_amount", "trip_distance"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

# Преобразуем данные
data_transformed = assembler.transform(df)

# Подготовим данные для модели
final_data = data_transformed.select('features', 'passenger_count_binary')


In [62]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- passenger_count_binary: integer (nullable = true)



Выполним разделение датасета на обучающую и тестовую выборки.

In [63]:
train_data, test_data = final_data.randomSplit([0.8, 0.2])

Закешируем сформированные датафреймы и проверим их объем.

In [64]:
train_data = train_data.cache()
test_data = test_data.cache()

print(f"Train dataset size: {train_data.count()}")
print(f"Test  dataset size: {test_data.count()}")

Train dataset size: 4829702


Test  dataset size: 1206409


#### Обучение модели

In [65]:
rf = RandomForestClassifier(labelCol="passenger_count_binary", featuresCol="features", numTrees=100)
cv_model = rf.fit(train_data)

#### Анализ обученной модели

Рассчитаем метрики на тестовом датасете.

In [66]:
# Делаем предсказания на тестовой выборк
predictions = cv_model.transform(test_data)

# Отображаем предсказания
predictions.select("passenger_count_binary", "prediction").show()

+----------------------+----------+
|passenger_count_binary|prediction|
+----------------------+----------+
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
|                     0|       0.0|
+----------------------+----------+
only showing top 20 rows



In [67]:
# Оценка модели с помощью метрик
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="passenger_count_binary", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="passenger_count_binary", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)

evaluator_precision = MulticlassClassificationEvaluator(labelCol="passenger_count_binary", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)

evaluator_recall = MulticlassClassificationEvaluator(labelCol="passenger_count_binary", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.9106646253467937
F1 Score: 0.8680854283440694
Precision: 0.8293100598580161
Recall: 0.9106646253467937


1. Precision (Точность):
   - Определяется как отношение истинно положительных результатов (True Positives, TP) к сумме истинно положительных и ложноположительных (False Positives, FP).
   - Эта метрика показывает, какую долю положительных предсказаний модель сделала правильно. Высокое значение precision означает, что модель не часто ошибается, когда предсказывает положительный класс.

2. Recall (Полнота):
   - Определяется как отношение истинно положительных результатов (TP) к сумме истинно положительных и ложноотрицательных (False Negatives, FN).
   - Recall показывает, какую долю реальных положительных примеров модель смогла обнаружить. Высокое значение recall означает, что модель мало упускает положительных примеров.

3. F1 Score:
   - Это среднее гармоническое между precision и recall.
   - F1 Score учитывает как false positives, так и false negatives, и является более сбалансированной метрикой, чем просто accuracy, особенно в случаях, когда классы несбалансированы. Высокое значение F1 Score указывает на то, что модель хорошо справляется с задачей обнаружения положительных классов.

In [ ]:
# Получаем датасет предсказаний
test_df_predictions = cv_model.transform(test_data)

# Извлекаем список колонок, устанавливаем цену на последнее место
right_columns_order = test_df_predictions.columns
right_columns_order.remove("passenger_count_binary")
right_columns_order.append("passenger_count_binary")

# Изменяем последовательность колонок и выводим датафрейм
test_df_predictions = test_df_predictions.select(*right_columns_order)
test_df_predictions.show()

#### Сохранение модели

Зададим директорию студента в `HDFS`, в которой будет сохранена обученная модель.

In [68]:
student_hdfs_folder = "gysynin_dmitry_data"

In [69]:
# Получаем имя пользователя
user_name = os.getenv("USER")

# Путь модели в HDFS
model_hdfs_path = f"hdfs:///user/{user_name}/{student_hdfs_folder}/models/rf-model-2"

# Сохраняем модель конвейера в HDFS
try:
    cv_model.save(model_hdfs_path)
    print(f"Модель успешно сохранена в \"{model_hdfs_path}\"")
except Exception as e:
    print(f"Ошибка при сохранении модели: {e}")

Модель успешно сохранена в "hdfs:///user/user0/gysynin_dmitry_data/models/rf-model-2"


Не забываем завершать `Spark`-сессию.

In [70]:
spark.stop()